In [1]:
import sys

sys.path.insert(0, "/home/elina/PycharmProjects/capstone_project")
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
from torchmetrics.classification import BinaryAUROC
from modules.model.main import ConvCatNet
from modules.data.preprocessing import PreProcess

import torch
import tenseal as ts
import numpy as np

from modules.encryption.keys import create_ctx

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
image_size = 64  # poly degree /2 should be bigger than image to columns for conv size
context, server_context = create_ctx()  # scale up as working with floats,


In [4]:
preprocessor = PreProcess(image_size=image_size)
train_dataloader, test_dataloader = preprocessor.preprocess()
net = ConvCatNet(image_size=image_size).to(device)
print(net)

  0%|          | 2/1920 [00:00<10:08,  3.15it/s]

ConvCatNet(
  (lrelu): LeakyReLU(negative_slope=0.1)
  (conv1): Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(3, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)


In [6]:
img = preprocessor.cuda_image_array.copy()[0][0]

conv1_weight = net.conv1.weight.data.view(
    -1, net.conv1.kernel_size[0],
    net.conv1.kernel_size[1]
).tolist()
conv1_bias = net.conv1.bias.data.tolist()
nk = img.tolist().copy()[0]

x_enc_0, windows_nb_0 = ts.im2col_encoding(
    context, nk, 3,
    3, 1
)
x_enc_0.shape


[61504]